#### Install ITensor

In [1]:
# using Pkg
# Pkg.add("ITensors")

#### Load ITensor

In [2]:
using ITensors

#### Index() and Itensor() Functions

- **Index()**: Define an Index represents a single tensor index with fixed dimension dim. 

- **Itensor()**: Constructor for an ITensor from a TensorStorage and a set of indices.

In [3]:
# ?ITensor #This is how you ask for help in Julia

In [4]:
# ?Index #This is how you ask for help in Julia

**Example of usage**:

In [5]:
i = Index(3)
j = Index(3)

A = ITensor(i,j) 

A[i=>1, j => 2] = 2 #This is how we set the elements inside the tensor (it does not matter the order).
# A[i=>1,j=>2] #This is how we can get the elements inside the tensor  (it does not matter the order).

println(A)

ITensor ord=2
Dim 1: (dim=3|id=369)
Dim 2: (dim=3|id=550)
NDTensors.Dense{Int64, Vector{Int64}}
 3×3
 0  2  0
 0  0  0
 0  0  0


**Matrix Example**:

Let's play a little bith with this. Consider the tensor product: $A_{ij}*B_{ik}$

In [6]:
i = Index(3)
j = Index(3)
k = Index(3)

A = ITensor(i,j) 
B = ITensor(i,k) 

for j_index in 1:dim(j)
    A[i=>1, j => j_index] = j_index
    A[i=>2, j => j_index] = 3 + j_index
    A[i=>3, j => j_index] = 6 + j_index
end

for k_index in 1:dim(j)
    B[i=>1, k => k_index] = (k_index)/10
    B[i=>2, k => k_index] = (3 + k_index)/10
    B[i=>3, k => k_index] = (6 + k_index)/10
end

In [7]:
println(A)

ITensor ord=2
Dim 1: (dim=3|id=480)
Dim 2: (dim=3|id=251)
NDTensors.Dense{Int64, Vector{Int64}}
 3×3
 1  2  3
 4  5  6
 7  8  9


In [8]:
println(B)

ITensor ord=2
Dim 1: (dim=3|id=480)
Dim 2: (dim=3|id=433)
NDTensors.Dense{Float64, Vector{Float64}}
 3×3
 0.1  0.2  0.3
 0.4  0.5  0.6
 0.7  0.8  0.9


I choose this values just because all of them are different. Let's try to contract the indixes using Itensor 

In [9]:
C = A*B

println(C)

ITensor ord=2
Dim 1: (dim=3|id=251)
Dim 2: (dim=3|id=433)
NDTensors.Dense{Float64, Vector{Float64}}
 3×3
 6.6   7.800000000000001   9.0
 7.8   9.3                10.8
 9.0  10.8                12.6


Let's try exactly the same without using Itensor, just Julia:

In [10]:
m, n = 3,3
A_matrix = fill(0.0, (3,3))
B_matrix = fill(0.0, (3,3))

for j_index in 1:3, i_index in 1:3
        A_matrix[i_index, j_index] = A[i=>i_index,j=>j_index]
end

for k_index in 1:3, i_index in 1:3
        B_matrix[i_index, k_index] = B[i=>i_index,k=>k_index]
end

In [11]:
A_matrix #i,j

3×3 Matrix{Float64}:
 1.0  2.0  3.0
 4.0  5.0  6.0
 7.0  8.0  9.0

In [12]:
B_matrix #k,i

3×3 Matrix{Float64}:
 0.1  0.2  0.3
 0.4  0.5  0.6
 0.7  0.8  0.9

Here we do not have an index, just matrixes. We must be careful thinking what index we can to contract, and do the operation that we really want.

It is not just A_matrix*B_matrix

In [13]:
A_matrix*B_matrix

3×3 Matrix{Float64}:
  3.0   3.6   4.2
  6.6   8.1   9.6
 10.2  12.6  15.0

We want $A_{ij}*B_{ik}$, and $(A*B)_{ik}$ = $A_{ij}*B_{jk}$, that is very different.

So $A_{ij}*B_{ik}$ = $A^{T}_{ji}*B_{ik}$ = $(A^{T}*B)_{jk}$

In [14]:
transpose(A_matrix)*B_matrix

3×3 Matrix{Float64}:
 6.6   7.8   9.0
 7.8   9.3  10.8
 9.0  10.8  12.6

We got the same result, but was harder without ITensor.

#### randomITensor() Function

Create an ITensor with normally-distributed random elements instead of specific values.

In [16]:
A = randomITensor(i,j,k)

ITensor ord=3 (dim=3|id=480) (dim=3|id=251) (dim=3|id=433)
NDTensors.Dense{Float64, Vector{Float64}}

In [17]:
println(A)

ITensor ord=3
Dim 1: (dim=3|id=480)
Dim 2: (dim=3|id=251)
Dim 3: (dim=3|id=433)
NDTensors.Dense{Float64, Vector{Float64}}
 3×3×3
[:, :, 1] =
  1.2561508181706398  -0.2177920198793908   -0.03070596014606092
  1.1144339434318813  -0.21862847720629294   0.025939044803507283
 -0.9866992758973254   0.4507056548007007    0.8945353015537075

[:, :, 2] =
  1.015123968741805    0.16671149792012843  -0.3117454541172909
 -0.33270681247748757  0.24523949792222458  -0.0850211521665733
 -0.5524822852231952   0.5652065184609159   -0.17789325375568701

[:, :, 3] =
 0.6992155613263914   1.022122817149885    1.566290225382175
 2.0637031350292743  -0.21203257343910267  1.5926067467613607
 0.5570677630357282  -0.540527449909668    1.7296023287204831


#### Linear Combinations of ITensors:

ITensors may also be subtracted and multiplied by scalars, including complex scalars, for example:

In [19]:
A = randomITensor(i,j,k)
B = randomITensor(k,i,j) 

ITensor ord=3 (dim=3|id=433) (dim=3|id=480) (dim=3|id=251)
NDTensors.Dense{Float64, Vector{Float64}}

In [21]:
C = 4*A - B/2 
D = A + 3.0im * B

ITensor ord=3 (dim=3|id=480) (dim=3|id=251) (dim=3|id=433)
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [26]:
println(D)

ITensor ord=3
Dim 1: (dim=3|id=480)
Dim 2: (dim=3|id=251)
Dim 3: (dim=3|id=433)
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 3×3×3
[:, :, 1] =
 -2.9013751848463154 - 3.8819026282880507im  -1.4455180194234138 + 1.3832114186685975im   -1.99062803328968 - 0.10210444816130013im
 -0.2470095274501079 + 1.2647459700429784im  0.46600432663353664 - 6.279371465151597im   0.8854809239647088 - 1.5768576991263634im
 -2.0734837288366648 - 1.3349230423045488im  -0.8086552468530479 - 4.792803237831798im    2.037856588550402 + 5.635122337034464im

[:, :, 2] =
 0.5581585906760147 + 1.2178158005700317im  -0.15031112346593073 + 3.8298152299336947im      1.0044986802148381 + 0.4525911163373254im
 0.5597267997059165 + 3.398428977304306im   -0.30137786441148967 + 0.055960661730226666im  0.004889856044485032 - 1.2944276402167187im
 -2.149821622775805 + 1.8802898453970975im   0.24487463219569075 + 5.328770512643749im     -0.11236884632551272 + 5.555622151057216im

[:, :, 3] =
  0.7130516396156691 - 0.14616

This is just possible because A and B have the same indexes:

In [23]:
l = Index(3)

A = randomITensor(i,j,k)
B = randomITensor(k,i,l) 

ITensor ord=3 (dim=3|id=433) (dim=3|id=480) (dim=3|id=234)
NDTensors.Dense{Float64, Vector{Float64}}

In [25]:
C = 4*A - B/2 

LoadError: You are trying to add an ITensor with indices:

((dim=3|id=433), (dim=3|id=480), (dim=3|id=234))

into an ITensor with indices:

((dim=3|id=480), (dim=3|id=251), (dim=3|id=433))

but the indices are not permutations of each other.
